In [3]:
!pip install -q -U google-genai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-jupyter-plugin 0.1.80 requires pydantic~=1.10.0, but you have pydantic 2.11.3 which is incompatible.
ydata-profiling 4.6.0 requires pydantic<2,>=1.8.1, but you have pydantic 2.11.3 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [26]:
from google import genai
from google.genai import types
from google.cloud import bigquery
import json

In [ ]:
PROJECT_ID = "qwiklabs-gcp-02-780f38a9daf1"
LOCATION = "us-central1"
MODEL = "gemini-2.0-flash-001"
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
)
SYSTEM_INSTRUCTION_TEXT = """
Refer to the data provided to answer questions about this city.
"""

bigquery_client = bigquery.Client()

def vector_search_bq(search_word):
    # Vector Search for Embeddings
    QUERY = (
        """
        SELECT query.query, base.question, base.answer
        FROM VECTOR_SEARCH(
          TABLE `qwiklabs-gcp-02-780f38a9daf1.challenge_2.aurora_bay_faqs_with_embeddings`, 'ml_generate_embedding_result',
          (
          SELECT text_embedding, content AS query
          FROM ML.GENERATE_TEXT_EMBEDDING(
          MODEL `challenge_2.embedding_model`,
          (SELECT @search_word AS content))
          ),
          top_k => 5, options => '{"fraction_lists_to_search": 0.01}')
          """
        )

    job_config = bigquery.QueryJobConfig(
        query_parameters = [
            bigquery.ScalarQueryParameter("search_word", "STRING", search_word)
        ])
    query_job = bigquery_client.query(QUERY, job_config = job_config)
    rows = [dict(row) for row in query_job.result()]
    if len(rows)==0:
        rows = []
    return rows



def start_chat():
    # Configure Gemini
    generate_content_config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 0.95,
        max_output_tokens = 8192,
        response_modalities = ["TEXT"],
        safety_settings = [types.SafetySetting(
          category="HARM_CATEGORY_HATE_SPEECH",
          threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
        ),types.SafetySetting(
          category="HARM_CATEGORY_DANGEROUS_CONTENT",
          threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
        ),types.SafetySetting(
          category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
          threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
        ),types.SafetySetting(
          category="HARM_CATEGORY_HARASSMENT",
          threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
        )],
        system_instruction=[types.Part.from_text(text=SYSTEM_INSTRUCTION_TEXT)],
      )    
    contents = []
    # Start chat
    print("Type 'exit' to stop chatting")
    while True:
        user_input = input("You: ")
        if user_input == "exit":
            break
        
        # Query Embeddings from User Input
        bigquery_content = vector_search_bq(user_input)
        
        reference = types.Part.from_function_call(
            name="reference",
            args={
                "json_data": json.dumps(bigquery_content)
            }
        )               
        contents.append(
            types.Content(
                role="user",
                parts=[
                    types.Part.from_text(text=user_input),
                    reference
                ]
            )
        )
        
        response = []
        print("Model: \n")
        # Model output
        for chunk in client.models.generate_content_stream(model = MODEL,contents = contents,config = generate_content_config):
            if chunk.text:
                print(chunk.text, end="")
                response.append(chunk.text)
        
        contents.append(
            types.Content(
                role="model",
                parts=[
                    types.Part.from_text(text=chunk.text)
                ]
            )
        )
        print("\n")


In [ ]:
start_chat()

Type 'exit' to stop chatting


You:  tax


Model: 

Okay, I'm ready. Please provide the data about the city and ask your questions. I will do my best to answer them using the provided information.




You:  tell me about rate


Model: 

Please specify what kind of rate you're interested in. I have information on business tax rates and noise ordinance hours.




You:  both


Model: 

Here's some information about business tax rates and noise ordinances in Aurora Bay:

*   **Business Tax Rates:** Business taxes include a 4% sales tax and a 2% local business tax on net profits, in addition to any applicable state taxes.
*   **Noise Ordinances:** Residential noise ordinances go into effect from 10 PM to 6 AM on weekdays and from 11 PM to 7 AM on weekends.

